# NSF Grant Data Quality Analysis

This notebook analyzes NSF grant JSON files to identify and export grants with data quality issues:
1. Grants with zero or null award amounts
2. Grants with empty or missing organizational directorate names

The analysis processes JSON files from 2015-2025 and creates CSV reports for data quality assessment.

## Import Required Libraries

In [1]:
import pandas as pd
import json
import os
from pathlib import Path
from tqdm import tqdm
import numpy as np

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## Set Up File Paths and Configuration

In [2]:
# Define base paths
data_dir = Path('./data/nsf')
export_dir = Path('./export/nsf')

# Years to process
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]

print(f"NSF data directory: {data_dir}")
print(f"Export directory: {export_dir}")
print(f"Years to process: {years}")

# Verify data directory exists
if data_dir.exists():
    print(f"✓ Data directory found")
    print(f"  Available year folders: {[d.name for d in data_dir.iterdir() if d.is_dir() and d.name.isdigit()]}")
else:
    print("✗ Data directory not found!")

NSF data directory: data/nsf
Export directory: export/nsf
Years to process: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
✓ Data directory found
  Available year folders: ['2022', '2025', '2024', '2023', '2015', '2017', '2019', '2021', '2020', '2018', '2016']


## Define Helper Functions for JSON Processing

In [3]:
def extract_grant_data(json_file_path):
    """
    Extract grant data from a JSON file.
    
    Args:
        json_file_path: Path to the JSON file
        
    Returns:
        dict: Extracted grant data or None if error
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            grant_data = json.load(f)
        
        # Extract year from file path (e.g., /data/nsf/2015/1062661.json -> 2015)
        year = int(json_file_path.parent.name)
        
        # Extract required fields with safe defaults
        extracted_data = {
            'awd_id': grant_data.get('awd_id', ''),
            'awd_amount': grant_data.get('awd_amount', None),
            'org_dir_long_name': grant_data.get('org_dir_long_name', ''),
            'awd_titl_txt': grant_data.get('awd_titl_txt', ''),
            'year': year,
            'file_path': str(json_file_path)
        }
        
        # Convert awd_amount to float if possible
        if extracted_data['awd_amount'] is not None:
            try:
                extracted_data['awd_amount'] = float(extracted_data['awd_amount'])
            except (ValueError, TypeError):
                extracted_data['awd_amount'] = None
        
        # Clean string fields
        for field in ['awd_id', 'org_dir_long_name', 'awd_titl_txt']:
            if extracted_data[field] and isinstance(extracted_data[field], str):
                extracted_data[field] = extracted_data[field].strip()
        
        return extracted_data
        
    except Exception as e:
        print(f"Error processing {json_file_path}: {e}")
        return None

def load_grants_from_year(year_dir):
    """
    Load all grants from a specific year directory.
    
    Args:
        year_dir: Path to year directory
        
    Returns:
        list: List of grant data dictionaries
    """
    grants = []
    json_files = list(year_dir.glob('*.json'))
    
    print(f"  Processing {len(json_files)} JSON files in {year_dir.name}...")
    
    for json_file in tqdm(json_files, desc=f"Loading {year_dir.name}"):
        grant_data = extract_grant_data(json_file)
        if grant_data:
            grants.append(grant_data)
    
    print(f"  Successfully loaded {len(grants)} grants from {year_dir.name}")
    return grants

print("Helper functions defined successfully!")

Helper functions defined successfully!


## Load and Process NSF JSON Files

In [4]:
# Load all grant data from all years
print("Loading NSF grant data from all years...")
print("=" * 50)

all_grants = []

for year in years:
    year_dir = data_dir / str(year)
    
    if year_dir.exists() and year_dir.is_dir():
        print(f"\nProcessing year {year}:")
        year_grants = load_grants_from_year(year_dir)
        all_grants.extend(year_grants)
    else:
        print(f"\nSkipping year {year}: Directory not found")

print(f"\n" + "=" * 50)
print(f"Total grants loaded across all years: {len(all_grants):,}")

# Convert to DataFrame
if all_grants:
    df_all_grants = pd.DataFrame(all_grants)
    print(f"DataFrame created with shape: {df_all_grants.shape}")
    print(f"Columns: {list(df_all_grants.columns)}")
    
    # Display basic info
    print(f"\nData types:")
    print(df_all_grants.dtypes)
    
    print(f"\nSample data:")
    print(df_all_grants.head())
else:
    print("No grants loaded!")

Loading NSF grant data from all years...

Processing year 2015:
  Processing 12848 JSON files in 2015...


Loading 2015: 100%|██████████| 12848/12848 [00:02<00:00, 4571.47it/s]


  Successfully loaded 12848 grants from 2015

Processing year 2016:
  Processing 12787 JSON files in 2016...


Loading 2016: 100%|██████████| 12787/12787 [00:02<00:00, 4670.94it/s]


  Successfully loaded 12787 grants from 2016

Processing year 2017:
  Processing 12309 JSON files in 2017...


...

  Successfully loaded 12022 grants from 2023

Processing year 2024:
  Processing 11687 JSON files in 2024...


Loading 2024: 100%|██████████| 11687/11687 [00:02<00:00, 5262.62it/s]


  Successfully loaded 11687 grants from 2024

Processing year 2025:
  Processing 9249 JSON files in 2025...


Loading 2025: 100%|██████████| 9249/9249 [00:01<00:00, 4650.59it/s]


  Successfully loaded 9249 grants from 2025

Total grants loaded across all years: 132,885
DataFrame created with shape: (132885, 6)
Columns: ['awd_id', 'awd_amount', 'org_dir_long_name', 'awd_titl_txt', 'year', 'file_path']

Data types:
awd_id                object
awd_amount           float64
org_dir_long_name     object
awd_titl_txt          object
year                   int64
file_path             object
dtype: object

Sample data:
    awd_id  awd_amount  \
0  1514971        70.0   
1  1504588     47267.0   
2  1515399      5070.0   
3  1544396    432000.0   
4  1501620     19702.0   

                                                  org_dir_long_name  \
0                                            Office Of The Director   
1                                    Directorate for STEM Education   
2                                            Office Of The Director   
3  Directorate for Computer and Information Science and Engineering   
4                               Directorate for 

## Filter Grants with Zero or Null Award Amounts

In [5]:
# Filter grants with zero or null award amounts
print("Filtering grants with zero or null award amounts...")

# Create filter for zero or null amounts
zero_null_amount_filter = (
    (df_all_grants['awd_amount'].isna()) |
    (df_all_grants['awd_amount'] == 0)
)

df_zero_null_amounts = df_all_grants[zero_null_amount_filter].copy()

print(f"Found {len(df_zero_null_amounts)} grants with zero or null award amounts")
print(f"Percentage of total grants: {len(df_zero_null_amounts) / len(df_all_grants) * 100:.3f}%")

if len(df_zero_null_amounts) > 0:
    print("\nBreakdown by year:")
    year_breakdown = df_zero_null_amounts['year'].value_counts().sort_index()
    for year, count in year_breakdown.items():
        print(f"  {year}: {count} grants")
    
    print("\nSample of grants with zero/null amounts:")
    display_cols = ['awd_id', 'awd_amount', 'org_dir_long_name', 'awd_titl_txt', 'year']
    print(df_zero_null_amounts[display_cols].head())
else:
    print("No grants found with zero or null award amounts.")

Filtering grants with zero or null award amounts...
Found 246 grants with zero or null award amounts
Percentage of total grants: 0.185%

Breakdown by year:
  2015: 10 grants
  2016: 20 grants
  2017: 13 grants
  2018: 28 grants
  2019: 19 grants
  2020: 32 grants
  2021: 33 grants
  2022: 35 grants
  2023: 22 grants
  2024: 25 grants
  2025: 9 grants

Sample of grants with zero/null amounts:
       awd_id  awd_amount  \
3101  1521151         0.0   
4347  1540003         0.0   
4373  1546092         0.0   
9363  1462233         0.0   
9915  1508593         0.0   

                                             org_dir_long_name  \
3101  Directorate for Technology, Innovation, and Partnerships   
4347                               Directorate for Engineering   
4373        Directorate for Mathematical and Physical Sciences   
9363  Directorate for Technology, Innovation, and Partnerships   
9915        Directorate for Mathematical and Physical Sciences   

                                 

## Filter Grants with Empty or Missing Directorate Names

In [6]:
# Filter grants with empty or missing directorate names
print("Filtering grants with empty or missing directorate names...")

# Create filter for empty or missing directorate names
empty_directorate_filter = (
    (df_all_grants['org_dir_long_name'].isna()) |
    (df_all_grants['org_dir_long_name'] == '') |
    (df_all_grants['org_dir_long_name'].str.strip() == '')
)

df_empty_directorate = df_all_grants[empty_directorate_filter].copy()

print(f"Found {len(df_empty_directorate)} grants with empty or missing directorate names")
print(f"Percentage of total grants: {len(df_empty_directorate) / len(df_all_grants) * 100:.3f}%")

if len(df_empty_directorate) > 0:
    print("\nBreakdown by year:")
    year_breakdown = df_empty_directorate['year'].value_counts().sort_index()
    for year, count in year_breakdown.items():
        print(f"  {year}: {count} grants")
    
    print("\nSample of grants with empty directorate names:")
    display_cols = ['awd_id', 'awd_amount', 'org_dir_long_name', 'awd_titl_txt', 'year']
    print(df_empty_directorate[display_cols].head())
else:
    print("No grants found with empty or missing directorate names.")

Filtering grants with empty or missing directorate names...
Found 41 grants with empty or missing directorate names
Percentage of total grants: 0.031%

Breakdown by year:
  2015: 3 grants
  2016: 4 grants
  2017: 4 grants
  2018: 3 grants
  2019: 2 grants
  2020: 4 grants
  2021: 7 grants
  2022: 12 grants
  2023: 2 grants

Sample of grants with empty directorate names:
        awd_id  awd_amount org_dir_long_name  \
612    1542281      3900.0                     
10179  1503577     18512.0                     
12679  1542282     15000.0                     
15462  1638466   4756507.0                     
15959  1654842    314635.0                     

                                                                                              awd_titl_txt  \
612    U.S. Department of State Diplomatic Security for the 2015 NSF/NSB Honorary Awards Ceremony and D...   
10179                                                                          Radio Communication Support   
12679  U

## Create Export Directory Structure

In [7]:
# Create export directory structure
print("Setting up export directory structure...")

# Create export/nsf directory if it doesn't exist
export_dir.mkdir(parents=True, exist_ok=True)

print(f"✓ Export directory created/verified: {export_dir}")
print(f"  Directory exists: {export_dir.exists()}")
print(f"  Directory is writable: {os.access(export_dir, os.W_OK)}")

# Define output file paths
zero_amounts_file = export_dir / 'nsf_grants_zero_null_amounts.csv'
empty_directorate_file = export_dir / 'nsf_grants_empty_directorate.csv'

print(f"\nPlanned output files:")
print(f"  1. Zero/null amounts: {zero_amounts_file}")
print(f"  2. Empty directorate: {empty_directorate_file}")

Setting up export directory structure...
✓ Export directory created/verified: export/nsf
  Directory exists: True
  Directory is writable: True

Planned output files:
  1. Zero/null amounts: export/nsf/nsf_grants_zero_null_amounts.csv
  2. Empty directorate: export/nsf/nsf_grants_empty_directorate.csv


## Export Filtered Data to CSV Files

In [8]:
# Export filtered data to CSV files
print("Exporting filtered data to CSV files...")

# Define columns to export
export_columns = ['awd_id', 'awd_amount', 'org_dir_long_name', 'awd_titl_txt', 'year']

# Export grants with zero or null amounts
if len(df_zero_null_amounts) > 0:
    df_zero_null_amounts[export_columns].to_csv(zero_amounts_file, index=False)
    print(f"✓ Exported {len(df_zero_null_amounts)} grants with zero/null amounts to:")
    print(f"  {zero_amounts_file}")
else:
    print("✓ No grants with zero/null amounts to export")

# Export grants with empty directorate names
if len(df_empty_directorate) > 0:
    df_empty_directorate[export_columns].to_csv(empty_directorate_file, index=False)
    print(f"✓ Exported {len(df_empty_directorate)} grants with empty directorate names to:")
    print(f"  {empty_directorate_file}")
else:
    print("✓ No grants with empty directorate names to export")

print(f"\nExport completed successfully!")

Exporting filtered data to CSV files...
✓ Exported 246 grants with zero/null amounts to:
  export/nsf/nsf_grants_zero_null_amounts.csv
✓ Exported 41 grants with empty directorate names to:
  export/nsf/nsf_grants_empty_directorate.csv

Export completed successfully!


## Display Summary Statistics

In [9]:
# Display comprehensive summary statistics
print("=" * 60)
print("NSF GRANT DATA QUALITY ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 OVERALL STATISTICS:")
print(f"  • Total grants processed: {len(df_all_grants):,}")
print(f"  • Years covered: {min(df_all_grants['year'])}-{max(df_all_grants['year'])}")
print(f"  • Total funding amount: ${df_all_grants['awd_amount'].sum():,.2f}")

print(f"\n🔍 DATA QUALITY ISSUES:")
print(f"  • Grants with zero/null amounts: {len(df_zero_null_amounts):,} ({len(df_zero_null_amounts)/len(df_all_grants)*100:.3f}%)")
print(f"  • Grants with empty directorate: {len(df_empty_directorate):,} ({len(df_empty_directorate)/len(df_all_grants)*100:.3f}%)")

# Show breakdown by year
print(f"\n📅 GRANTS BY YEAR:")
year_stats = df_all_grants.groupby('year').agg({
    'awd_id': 'count',
    'awd_amount': ['sum', 'mean']
}).round(2)
year_stats.columns = ['Count', 'Total_Funding', 'Mean_Amount']
print(year_stats)

print(f"\n📁 EXPORTED FILES:")
if len(df_zero_null_amounts) > 0:
    print(f"  ✓ {zero_amounts_file.name}")
if len(df_empty_directorate) > 0:
    print(f"  ✓ {empty_directorate_file.name}")

print(f"\n🎉 Analysis completed successfully!")
print("=" * 60)

NSF GRANT DATA QUALITY ANALYSIS SUMMARY

📊 OVERALL STATISTICS:
  • Total grants processed: 132,885
  • Years covered: 2015-2025
  • Total funding amount: $78,288,497,999.00

🔍 DATA QUALITY ISSUES:
  • Grants with zero/null amounts: 246 (0.185%)
  • Grants with empty directorate: 41 (0.031%)

📅 GRANTS BY YEAR:
      Count  Total_Funding  Mean_Amount
year                                   
2015  12848   6.056397e+09    471388.30
2016  12787   7.744449e+09    605650.19
2017  12309   6.847584e+09    556307.09
2018  12684   9.049235e+09    713437.00
2019  12180   7.120009e+09    584565.58
2020  13041   7.502284e+09    575284.43
2021  12166   8.193757e+09    673773.30
2022  11912   7.269309e+09    610507.22
2023  12022   7.348115e+09    611222.32
2024  11687   6.385263e+09    546356.06
2025   9249   4.772096e+09    515958.02

📁 EXPORTED FILES:
  ✓ nsf_grants_zero_null_amounts.csv
  ✓ nsf_grants_empty_directorate.csv

🎉 Analysis completed successfully!
